<p style="background:#f4f4f4; padding:5px; margin-left:-5px;margin-bottom:0px">
Informática - 1º de Física
<br>
<strong>Introducción a la Programación</strong>
</p>

# Caos determinista

*A.Ruiz, J. Abellán*

En este notebook se reproducen los experimentos descritos en el artículo *Simple mathematical models with very complicated dynamics* ([Robert May, 1976](http://abel.harvard.edu/archive/118r_spring_05/docs/may.pdf)).

## Visualización

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sys
if 'pyodide' in sys.modules:
    %pip install ipywidgets

In [ ]:
X = np.linspace(0,1,1000)

def f(l,x):
    return l*x*(1-x)


def iterada(n,f):
    def g(l,x):
        z = x
        for k in range(n):
            z = f(l,z)
        return z
    return g

In [ ]:
def fun(lam=3,steps=30,niter=1,x0=0.2):
    plt.figure(figsize=(6,6))
    #x0 = 1-1/lam + 0.001
    g = iterada(niter,f)
    plt.plot(X,g(lam,X))
    plt.plot(X,X,color='gray',ls='dashed');
    x  = [x0]
    xp = [x0]
    yp = [0]
    nx = x0
    for k in range(steps):
        nx = g(lam,nx)
        x  += [nx]
        xp += [xp[-1],nx]          # añadimos dos tramos, uno  vertical hacia la función
        yp += [nx,nx]              # y otro horizontal hacia la bisectriz 
    plt.plot(xp,yp,color='red',lw=0.5)
    plt.plot(x,np.zeros(len(x)),'.',color='blue',alpha=0.15, markersize=15)
    #plt.title(str(nx))

In [ ]:
fun(2, 2, 1, 0.2)

arrow = arrowprops=dict(color='gray',arrowstyle="->", connectionstyle="arc3")

plt.annotate('$x_0$', xy=(0.21,0.01), xytext=(0.25,0.1), arrowprops=arrow);
plt.annotate('$x_1=\lambda x_0 (1-x_0)$', xy=(0.19,0.32), xytext=(0.0,0.4), arrowprops=arrow, fontsize=13);
plt.annotate('$x_1$', xy=(0.33,0.01), xytext=(0.4,0.2), arrowprops=arrow);
plt.annotate('', xy=(0.33,0.32), xytext=(0.4,0.22), arrowprops=arrow);
plt.annotate('$x_2$', xy=(0.44,0.01), xytext=(0.5,0.25), arrowprops=arrow);
plt.annotate('', xy=(0.44,0.43), xytext=(0.5,0.27), arrowprops=arrow);
plt.annotate('$x_2=\lambda x_1 (1-x_1)$', xy=(0.31,0.44), xytext=(0.1,0.6), arrowprops=arrow,fontsize=13);
plt.legend(["$f(x)=\lambda x (1-x)$",'$f(x)=x$']); plt.xlabel('$x_k$'); plt.ylabel('$x_{k+1}$');

In [ ]:
fun(2.5, 100, x0=0.1)

In [ ]:
fun(2.5, 100, x0=0.95)

In [ ]:
fun(3.83, 100, x0=0.52)

In [ ]:
fun(3.83, 100, 3, 0.52)

### Visualización interactiva

In [ ]:
from ipywidgets import interact

@interact(lam=(2.,4,0.01),steps=(0,30),niter=(1,16),x0=(0.,1,0.01))
def ifun(lam=3,steps=30,niter=1,x0=0.2):
    fun(lam,steps,niter,x0)
    plt.show()

### Diagrama de bifurcación

Generamos una secuencia de longitud deseada.

In [ ]:
def mkSec(lam, x0=0.5, n=10000):
    x = np.zeros(n)
    x[0] = x0
    for k in range(1,n):
        x[k] = f(lam,x[k-1])
    return x

In [ ]:
mkSec(3.5, x0=0.1, n=30)

Luego dejamos avanzar la secuencia 'drop' elementos, y nos quedamoscon los 'n' últimos, donde se supone que ya se ha estabilizado en un ciclo.

In [ ]:
def limit(lam, x0=0.5, drop=1000, n=100):
    return mkSec(lam, x0, drop+n)[-n:]

Lo comprobamos con un $\lambda$, partiendo de dos valores iniciales distintos. (Agrupamos de 4 en 4 para mostrar visualmente el ciclo. Salen los mismos valores, aunque en orden distinto.)

In [ ]:
limit(3.5,0.5, n=20).reshape(-1,4)

In [ ]:
limit(3.5,0.2, n=20).reshape(-1,4)

Para construir el diagrama de bifurcación generamos el ciclo final de la secuencia para sucesivos valores de $\lambda$, y se dibuja cada conjunto en la misma posición horizontal.

In [ ]:
def bifurcación(l1,l2):
    lams = np.linspace(l1,l2,200)
    for l in lams:
        y = limit(l,n=100)
        x = l*np.ones(len(y))
        plt.plot(x,y,'.',color='red',markersize=1)

In [ ]:
plt.figure(figsize=(8,5))
bifurcación(2.8,3.9)
plt.xlabel("$\lambda$"); plt.ylabel("$x_\infty$");

Hacemos zoom en un par de zonas:

In [ ]:
plt.figure(figsize=(8,5))
bifurcación(3.723,3.724)
plt.xlabel("$\lambda$"); plt.ylabel("$x_\infty$");

In [ ]:
plt.figure(figsize=(8,5))
bifurcación(3.735,3.745)
plt.xlabel("$\lambda$"); plt.ylabel("$x_\infty$");

### Exponente de Lyapunov

In [ ]:
import itertools as it

def f(l,x):
    return l*x*(1-x)

def deriv(l,x):
    return l*(1-2*x)

def mkSecxd(lam,x0=0.5):
    x = x0
    while True:
        nx = f(lam,x)
        d  = deriv(lam,nx)
        yield (nx,d)

def lyapunov(lam,x0=0.5, n=1000):
    sec = it.islice(mkSecxd(lam),n)
    l = [np.log(abs(d)) for _,d in sec]
    r = np.mean(l)
    return r

In [ ]:
lam = np.linspace(2.5,4,100)
plt.plot(lam,[lyapunov(l,0.2) for l in lam])
plt.plot([lam[0],lam[-1]],[1,1],color='red');

### Aleatoriedad *vs* caos

In [ ]:
def exper(x):
    plt.figure(figsize=(8,8))
    plt.subplot(2,2,1)
    plt.plot(x,'.')
    plt.xlabel('$k$')
    plt.ylabel('$x_k$')
    plt.subplot(2,2,3)
    plt.plot(x[:-1],x[1:],'.');
    plt.xlabel('$x_k$')
    plt.ylabel('$x_{k+1}$')
    plt.axis('equal');
    plt.subplot(2,2,2)
    plt.hist(x,orientation='horizontal');

In [ ]:
r = np.random.rand(1000)

exper(r)

In [ ]:
s = np.array(list(it.islice(mkSec(4,0.1),1000)))

exper(s)

In [ ]:
s = np.array(list(it.islice(mkSec(3.738,0.1),1000)))

exper(s)

### Solución analítica

Mediante un enfoque combinado de cálculo simbólico y numérico.

In [ ]:
import numpy as np
from sympy import *
from sympy.abc import alpha, beta, gamma, delta, epsilon

init_printing(pretty_print=True)

from IPython.display import display

In [ ]:
lamda,x = symbols(['lambda','x'])

In [ ]:
def f(x): return lamda * x * (1-x)

def iterada(n,f):
    def g(x):
        z = x
        for k in range(n):
            z = f(z)
        return z
    return g

In [ ]:
(f(f(x))-x).expand()

In [ ]:
solve(Eq(x, f(f(x)).expand()),x)

In [ ]:
def cortes(it,l,verbose=False):
    h = iterada(it,f)(x)
    if verbose:
        display(h)
    fun = poly(h.subs(lamda,l),x)
    der = diff(fun,x)
    g = fun-x
    sols = np.roots(g.coeffs())
    r = [(v.real, der.subs(x,v.real)) for v in sols if abs(v.imag) < 1E-6]
    return r

In [ ]:
cortes(3,3.8,verbose=True)

In [ ]:
cortes(8,3.7)